In [1]:
import pysam
import pandas as pd
import numpy as np
import re
import os
import glob

# Load the fasta

In [2]:
def fasta_to_dict(fasta_file):
  fasta_dict = {}
  with pysam.FastaFile(fasta_file) as fasta:
    for ref in fasta.references:
      fasta_dict[ref] = fasta.fetch(ref)
  return fasta_dict

fpath = "/nfs/turbo/umms-indikar/shared/projects/hybrid_reprogramming/data/adapter_search/search.fasta"
fasta = fasta_to_dict(fpath)
fasta

{'polyA': 'AAAAAAAAAAA',
 'polyT': 'TTTTTTTTTTT',
 '10x_left_adapter': 'CTACACGACGCTCTTCCGATCT',
 '10x_right_adapter': 'CCCATGTACTCTGCGTTGATACCACTGCTT',
 'biolegend_1': 'GTCAACTCTTTAGCG',
 'biolegend_2': 'TGATGGCCTATTGGG',
 'biolegend_3': 'TTCCGCCTCTCTTTG',
 'biosig': 'CAGCACTTGCCTGTCGCTCTATCTTC'}

# Load demux results

In [3]:
fpath = "/scratch/indikar_root/indikar1/shared_data/hybrid_reprogramming/demultiplex/run1.putative_bc.csv"
df = pd.read_csv(fpath)
df = df.set_index('read_id')
df['demuxed'] = ~df['putative_bc'].isna()
print(f"{df.shape=}")
df.head()

df.shape=(5930000, 7)


,putative_bc,putative_bc_min_q,putative_umi,polyT_end,pre_bc_flanking,post_umi_flanking,demuxed
read_id,,,,,,,
be606770-65a1-40be-89a6-c7056ce4d70e,NaN,NaN,NaN,NaN,NaN,NaN,False
74def5ea-567d-46dc-a16c-0cfa235ecd3f,NaN,NaN,NaN,NaN,NaN,NaN,False
4e43e793-a9d2-485b-8010-11fcfa3820cf,NaN,NaN,NaN,NaN,NaN,NaN,False
e243188c-0b57-4b67-9337-aa2a91a741a4,CAAGGATAGCTAATGT,18.0,GGTATCGGTGGA,-174.0,GATCT,TTTTT,True
96a72289-6bbb-4123-9489-4645e1535cb2,TGGACAAGTAGGTCAA,37.0,CTAGGCCCATCT,-125.0,GATCT,TTTTT,True


In [4]:
df['demuxed'].value_counts(normalize=True)

demuxed
False    0.735721
True     0.264279
Name: proportion, dtype: float64

# Load Fastq

In [5]:
dpath = "/nfs/turbo/umms-indikar/shared/projects/hybrid_reprogramming/data/fastq/gene_expression/2024-12-09-HybridExp2-MYOD-PRRX1/2024-12-04-Hybrid-GEX-Single-Cell-1/fastq_pass/"
file_list = glob.glob(f"{dpath}/*fastq.gz")
print(len(file_list))

[os.path.basename(x) for x in file_list[:3]]

262


['PAS86317_pass_c40efca7_c272b00d_26.fastq.gz',
 'PAS86317_pass_c40efca7_c272b00d_161.fastq.gz',
 'PAS86317_pass_c40efca7_c272b00d_34.fastq.gz']

In [6]:
fpath = np.random.choice(file_list, 1)[0]
print(f"File: {os.path.basename(fpath)}")

poly_length = 8
fastq = pysam.FastqFile(fpath)

results = []
for read in fastq:

    row = {
        'read_name' : read.name,
        'seq' : read.sequence,
    }
    results.append(row)
    
results = pd.DataFrame(results)
print(f"{results.shape=}")
results.head()

File: PAS86317_pass_c40efca7_c272b00d_1.fastq.gz
results.shape=(172839, 2)


,read_name,seq
0,c8ae3cba-7890-4728-923a-d6bb02f43178,GTGTTTTGCATGTACTTCGTTCAGTTACGTATTGCTCAGCTTTCTG...
1,58295a98-d889-4c84-a85a-89364485de33,TGCTCCGTTACTTCGTTCAGTTACGTATTGCTCAGCTTTCTGTTGG...
2,50b68cb0-a476-42f8-b3c5-c34c9ba60f8e,TTTTTTTTGCATGTACTTCGTTCAGTTACGTATTGCTCTTGCCTGT...
3,0b692242-9af6-482e-9cd7-06952be461e9,CTTTTATTCCTGCCTCGACTACATTACGTATTGCTGGTGCTGATAT...
4,75cafcb9-3b50-461f-88fa-fa2f0e2a49a4,ATGTTGTGTAGCCTTGACTACAAGTTACGTATTGCTCAGCTTTCTG...


# Subset the demux results

In [7]:
reads = df.loc[results['read_name'].values]
reads = reads.reset_index()
reads = pd.concat([reads, results], axis=1)
print(f"{reads.shape=}")
print()
print(reads['demuxed'].value_counts(normalize=True))
print()
reads.head()

KeyError: "None of [Index(['c8ae3cba-7890-4728-923a-d6bb02f43178',\n       '58295a98-d889-4c84-a85a-89364485de33',\n       '50b68cb0-a476-42f8-b3c5-c34c9ba60f8e',\n       '0b692242-9af6-482e-9cd7-06952be461e9',\n       '75cafcb9-3b50-461f-88fa-fa2f0e2a49a4',\n       '319d20f4-7b2b-41c7-ae86-126c4cb2f640',\n       'eba1656a-8280-449c-b23e-3b14b5dd2ec5',\n       'be789eee-6a99-42f3-829a-dc8a96e61437',\n       'a7034ed6-e4ef-4a48-b9c7-4c26dca59b30',\n       '20969a45-1a06-4dd7-9a16-9c654a05e42f',\n       ...\n       '3ccadbed-28b4-4d3b-b317-75aa5dd25c71',\n       '34c38215-868e-4822-aad3-0df9af229581',\n       '8088d94d-189b-4c93-86b4-a84506f66537',\n       'bd732871-fd20-4188-8ee8-806e0c74a712',\n       '0881c72a-a224-428e-9568-a635ec438a41',\n       'e7ce4391-cd41-49ea-a07f-9853347e2c4a',\n       'c2a86d8a-3a28-46c0-a347-f457f0c86538',\n       'd4c3231a-d601-4e78-ba0d-218228602e32',\n       'eeb5ba5d-fd48-4966-b732-8269c4427b9f',\n       '1b88a07c-71d7-4400-8f23-49a13460ca4d'],\n      d

In [ ]:
break

In [ ]:
break

In [ ]:
break

In [ ]:
def reverse_complement(seq):
  """
  This function takes a DNA sequence as input and returns its reverse complement.

  Args:
    seq: The DNA sequence to be reversed and complemented.

  Returns:
    The reverse complement of the input sequence.
  """
  complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
  return "".join(complement.get(base, base) for base in reversed(seq))

In [ ]:
# randomly sample a fastq file

fpath = np.random.choice(file_list, 1)[0]
print(f"File: {os.path.basename(fpath)}")

poly_length = 8
polya = "A" * poly_length
polyt = "T" * poly_length
n_records = 5
fastq = pysam.FastqFile(fpath)

count = -1
for read in fastq:
    count += 1
    if count == n_records:
        break

    read_id = read.name
    seq = read.sequence
    print(f"{read_id=}")
    for idx, val in df.loc[read_id].items():
        print(f"{idx}: {val}")
    print(seq)

    print()


In [ ]:
rec

In [ ]:
?pysam.FastxFile

In [ ]:
fpath = "/scratch/indikar_root/indikar1/shared_data/hybrid_reprogramming/fastq/run1.raw.fastq.gz"
poly_length = 8
polya = "A" * poly_length
polyt = "T" * poly_length
n_records = 2
fastq = pysam.FastqFile(fpath)

# Define the sequences to search for with potential mismatches
polya_seq = "GATGTGCTGCGAGAAGGCTAGA"
polyt_seq = "CTACACGACGCTCTTCCGATCT"

def highlight_poly(sequence, poly):
    """Highlights poly-A or poly-T tails in a sequence with red color."""
    if poly in sequence:
        return sequence.replace(poly, f"\033[91m{poly}\033[0m")
    return sequence

def highlight_sequence(sequence, target_seq, color="red", mismatches=2):
    """Highlights the target sequence with the specified color and number of mismatches allowed.

    Args:
      sequence: The sequence to search in.
      target_seq: The sequence to search for.
      color: The color to highlight the sequence with. 
             Can be one of "blue", "red", "green", "yellow", "magenta", "cyan".
      mismatches: The number of mismatches allowed.

    Returns:
      The sequence with the target sequence highlighted.
    """

    # Define ANSI escape codes for colors
    color_codes = {
        "blue": "\033[94m",
        "red": "\033[91m",
        "green": "\033[92m",
        "yellow": "\033[93m",
        "magenta": "\033[95m",
        "cyan": "\033[96m",
    }

    if color not in color_codes:
        raise ValueError(f"Invalid color: {color}")

    # Use regex to find the sequence with the specified number of mismatches
    match = re.search(f"({target_seq}){{s<={mismatches}}}", sequence)
    if match:
        # Get the matched sequence
        matched_seq = match.group(0)
        # Highlight the matched sequence with the specified color
        highlighted_seq = f"{color_codes[color]}{matched_seq}\033[0m"
        # Replace the matched sequence in the original sequence with the highlighted sequence
        return sequence.replace(matched_seq, highlighted_seq)
    return sequence

count = -1
for read in fastq:
    seq = read.sequence
    if not polya in seq or polyt in seq:
        continue

    count += 1
    if count == n_records:
        break

    print(f"{read.name=}")

    # seq_search = highlight_poly(seq, polya)
    # seq_search = highlight_poly(seq_search, polyt)
    seq_search = highlight_sequence(seq, "AAAAATAATA")
    print(seq_search)
    
    # print(highlight_sequence(seq, polya_seq))



    print()